# Running the ICML 2018 Experiments

In this series of notebooks, we will replicate and analyze the ICML 2018 experiments that were used for benchmarking in <a href="http://proceedings.mlr.press/v80/falkner18a.html" target="_blank">BOHB (Falkner et al. 2018)</a>.
In addition to <a href="https://github.com/automl/HpBandSter" target="_blank">HpBandSter</a>, we will use <a href="https://github.com/automl/CAVE" target="_blank">CAVE</a> to analyze and visualize the optimization process.

## About this experiment

### Cartpole

"A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center." (<a href="https://gym.openai.com/envs/CartPole-v0/" target="_blank">openAI gym description</a>)

### Installation requirements

To run the experiments, please install the <a href="https://github.com/automl/BOHBsCAVE/blob/master/examples/icml_2018_experiments/requirements.txt" target="_blank">requirements</a>, e.g. `pip install -r examples/icml_2018_experiments/requirements.txt`.

## Run the experiment


This examples ships with all the code necessary to reproduce the experiment. Because it takes a few days to generate the data, the results of the optimization are provided in `examples/icml_2018_experiments/opt_results/`. If you want to generate the data (from examples/icml_2018_experiments), just run:

In [ ]:
! python scripts/run_experiment.py --exp_name cartpole --num_iterations 1 --min_budget 1 --max_budget 3 --n_workers 1

You can use this script to generate the optimization data for a variety of optimizers, the results will (by default) be stored in `examples/icml_2018_experiments/EXPERIMENT/OPTIMIZER`. Use `python run_experiment --help` to see how to use the script.
If you have access to a cluster, take a look at the scripts provided for cluster computation on a SLURM cluster at `examples/icml_2018_experiments/scripts/cluster/`.

## Using the results in CAVE

### Instantiate CAVE

Analyzing the optimization results with CAVE is very straight-forward. If you want to use CAVE interactively in a notebook, set `show_jupyter=True`. Specify which optimization you want to analyze via the `folders` argument and specify `file_format==SMAC3` or `file_format==BOHB`, depending on which optimizer was used for the results. To analyze how BOHB optimized the cartpole-problem, run:

In [ ]:
from cave.cavefacade import CAVE

cave = CAVE(folders=["opt_results/cartpole/bohb"],
            output_dir="CAVE_reports/cartpole_notebook",  # output for debug/images/etc
            ta_exec_dir=["."],                            # Only important for SMAC-results
            file_format='BOHB',                           # BOHB or SMAC3
            verbose_level='OFF',
            show_jupyter=True,
           )

To generate the HTML-report you can use the `analyze`-method. The report is located in `output_dir/report.html`, so in this case in `CAVE_reports/cartpole_notebook/report.html`.

In [ ]:
%%capture
cave.analyze()
! firefox CAVE_reports/cartpole_notebook/report.html

CAVE is fully compatible with Jupyter notebooks. You can invoke the individual analysis methods as follows.

The most interesting plot for BOHB might be a visualization of the learning curves:

In [ ]:
cave.bohb_learning_curves();

In [ ]:
cave.overview_table();

For each budget, we can list the cost over incumbents:

In [ ]:
cave.bohb_incumbents_per_budget();

For parameter-importance analysis, CAVE uses <a href="https://github.com/automl/ParameterImportance" target="_blank">PIMP</a> , a package that provides multiple approaches to parameter-importance analysis. We can easily invoke them via CAVE, of course. To estimate the importance, random forests are used to predict performances of configurations that were not executed. This is difficult for big budgets with few configurations.

We can access the individual budgets via the 'run'-keyword-argument of each analysis-method.

In [ ]:
cave.cave_fanova(run='budget_3');

In [ ]:
cave.local_parameter_importance(run='budget_3');

For each budget, we can compare the different parameter-importance-methods that have already been run:

In [ ]:
cave.pimp_comparison_table(run='budget_3');

To analyze BOHB's behaviour, we can check out the configurator footprint, cost-over-time and parallel coordinated parameters:

In [ ]:
cave.configurator_footprint(use_timeslider=True, num_quantiles=5);

In [ ]:
cave.cost_over_time();

In [ ]:
cave.parallel_coordinates(run='budget_1');